## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-25-06-47-23 +0000,bbc,Venezuela's Maduro says US 'fabricating war' a...,https://www.bbc.com/news/articles/c891gzx7xn4o...
1,2025-10-25-06-34-23 +0000,nypost,FBI Director Kash Patel calls Philadelphia dru...,https://nypost.com/2025/10/25/us-news/fbi-indi...
2,2025-10-25-06-12-15 +0000,startribune,Takeaways: Lakers flatten Timberwolves in ‘emb...,https://www.startribune.com/takeaways-lakers-f...
3,2025-10-25-06-11-27 +0000,bbc,'I left Wales for England to access free child...,https://www.bbc.com/news/articles/c2lpnr8n8rpo...
4,2025-10-25-05-32-06 +0000,nypost,Mississippi substitute teacher fired on first ...,https://nypost.com/2025/10/25/us-news/mississi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
116,trump,55
348,trade,14
266,canada,14
233,anti,12
621,inflation,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
106,2025-10-24-21-43-10 +0000,nypost,Ontario to pause anti-tariff Reagan ads after ...,https://nypost.com/2025/10/24/us-news/ontario-...,142
146,2025-10-24-19-57-00 +0000,wsj,Ontario to Pause Anti-Tariff Ad Campaign After...,https://www.wsj.com/world/americas/ontario-to-...,139
159,2025-10-24-19-27-00 +0000,wsj,Ontario to Pause Anti-Tariff Commercial After ...,https://www.wsj.com/world/americas/ontario-to-...,124
165,2025-10-24-18-51-56 +0000,wapo,Trump cancels trade negotiations with Canada o...,https://www.washingtonpost.com/world/2025/10/2...,117
95,2025-10-24-22-03-08 +0000,nyt,"Trump Decries Anti-Tariff Ad From Ontario, Can...",https://www.nytimes.com/2025/10/24/us/politics...,116


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
106,142,2025-10-24-21-43-10 +0000,nypost,Ontario to pause anti-tariff Reagan ads after ...,https://nypost.com/2025/10/24/us-news/ontario-...
228,79,2025-10-24-15-22-00 +0000,wsj,Annual inflation heated up slightly in Septemb...,https://www.wsj.com/economy/consumer-price-ind...
214,56,2025-10-24-16-12-41 +0000,latimes,"U.S. sanctions Colombia's president, deploys a...",https://www.latimes.com/world-nation/story/202...
235,47,2025-10-24-14-59-00 +0000,wsj,The extended government shutdown has put colle...,https://www.wsj.com/economy/shutdown-puts-octo...
61,45,2025-10-24-23-42-29 +0000,nypost,Hunter Biden claims he didn’t think ‘laptop fr...,https://nypost.com/2025/10/24/us-news/hunter-b...
126,43,2025-10-24-20-56-50 +0000,latimes,Video shows security opening fire on U-Haul tr...,https://www.latimes.com/california/story/2025-...
189,43,2025-10-24-17-43-51 +0000,nypost,Influential House Minority Leader Hakeem Jeffr...,https://nypost.com/2025/10/24/us-news/hakeem-j...
181,41,2025-10-24-17-59-23 +0000,nypost,Rubio warns West Bank annexation would threate...,https://nypost.com/2025/10/24/world-news/rubio...
254,40,2025-10-24-13-54-28 +0000,nypost,"China, Russia use ‘asymmetric advantage,’ unle...",https://nypost.com/2025/10/24/business/china-r...
154,33,2025-10-24-19-35-06 +0000,latimes,Mega Millions alcanza los 680 millones de dóla...,https://www.latimes.com/espanol/california/art...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
